In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Lambda
from tensorflow.keras.layers import Conv3D, MaxPooling3D, ZeroPadding3D
from tensorflow.keras.optimizers import Adam

In [2]:
def get_3dConv_model():
    model = Sequential()

    # 1st layer group
    model.add(Conv3D(64, (3, 3, 3), activation="relu",name="conv1", 
                     input_shape=(10,150,150,3),
                     strides=(1, 1, 1), padding="same"))  
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid"))

    # 2nd layer group  
    model.add(Conv3D(128, (3, 3, 3), activation="relu",name="conv2", 
                     strides=(1, 1, 1), padding="same"))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="pool2", padding="valid"))

    # 3rd layer group   
    model.add(Conv3D(256, (3, 3, 3), activation="relu",name="conv3a", 
                     strides=(1, 1, 1), padding="same"))
    #model.add(Conv3D(256, (3, 3, 3), activation="relu",name="conv3b", 
    #                 strides=(1, 1, 1), padding="same"))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool3", padding="valid"))

    # 4th layer group  
    model.add(Conv3D(256, (3, 3, 3), activation="relu",name="conv4a", 
                     strides=(1, 1, 1), padding="same"))   
    #model.add(Conv3D(256, (3, 3, 3), activation="relu",name="conv4b", 
    #                 strides=(1, 1, 1), padding="same"))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="pool4", padding="valid"))

    # 5th layer group  
    model.add(Conv3D(512, (3, 3, 3), activation="relu",name="conv5a", 
                     strides=(1, 1, 1), padding="same"))   
    #model.add(Conv3D(512, (3, 3, 3), activation="relu",name="conv5b",
    #                  strides=(1, 1, 1), padding="same"))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool5", padding="valid"))
    model.add(Flatten())
                     
    # FC layers group
    model.add(Dense(2, activation='softmax', name='fc8'))

    model.compile(
        optimizer=Adam(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [3]:
my_model = get_3dConv_model()
my_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 10, 150, 150, 64)  5248      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 10, 75, 75, 64)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 10, 75, 75, 128)   221312    
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 5, 37, 37, 128)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 5, 37, 37, 256)    884992    
_________________________________________________________________
pool3 (MaxPooling3D)         (None, 5, 18, 18, 256)    0         
_________________________________________________________________
conv4a (Conv3D)              (None, 5, 18, 18, 256)    1

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [6]:
torch.cuda.is_available()

True

In [32]:
class DeepFakeDetector(torch.nn.Module):
    
    def __init__(self):
        super(DeepFakeDetector, self).__init__()
        # 1st 3D Conv layer
        self.conv3D_1 = nn.Conv3d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.maxpool3D_1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
        
        # 2nd 3D Conv layer
        self.conv3D_2 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.maxpool3D_2 = nn.MaxPool3d(kernel_size=2, stride=2)
        
        # 3rd 3D Conv layer
        self.conv3D_3a = nn.Conv3d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.conv3D_3b = nn.Conv3d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.maxpool3D_3 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
        
        # 4th 3D Conv layer
        self.conv3D_4a = nn.Conv3d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.conv3D_4b = nn.Conv3d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.maxpool3D_4 = nn.MaxPool3d(kernel_size=2, stride=2)
        
        # 5th 3D Conv layer
        self.conv3D_5a = nn.Conv3d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.conv3D_5b = nn.Conv3d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.maxpool3D_5 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
        
        # Fully Connected layer
        self.fc_1 = nn.Linear(in_features=16384, out_features=512)
        self.output = nn.Linear(in_features=512, out_features=2)
        
    def forward(self, x):
        out = F.relu(self.conv3D_1(x))
        out = self.maxpool3D_1(out)
        
        out = F.relu(self.conv3D_2(out))
        out = self.maxpool3D_2(out)
        
        out = F.relu(self.conv3D_3a(out))
        out = F.relu(self.conv3D_3b(out))
        out = self.maxpool3D_3(out)
        
        out = F.relu(self.conv3D_4a(out))
        out = F.relu(self.conv3D_4b(out))
        out = self.maxpool3D_4(out)
        
        out = F.relu(self.conv3D_5a(out))
        out = F.relu(self.conv3D_5b(out))
        out = self.maxpool3D_5(out)
        
        out = torch.flatten(out, start_dim=1)
        
        out = F.relu(self.fc_1(out))
        out = self.output(out)
        out = nn.Softmax(dim=1)(out)
        
        return out

In [33]:
m = DeepFakeDetector()
a = torch.zeros(2, 3, 10, 150, 150)
b = m(a)
b

tensor([[0.4988, 0.5012],
        [0.4988, 0.5012]], grad_fn=<SoftmaxBackward>)

In [40]:
from torchvision import transforms
from PIL import Image as pil_image
import cv2

t = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize([0.5] * 3, [0.5] * 3)
])

In [49]:
a = np.zeros((150, 150, 3), dtype=np.float32)
print(a.shape)
a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
print(a.shape)
b = pil_image.fromarray(np.uint8(a))
print(b.size)
b = t(b)
b.shape

(150, 150, 3)
(150, 150, 3)
(150, 150)


torch.Size([3, 299, 299])

In [54]:
e = torch.randn(2, 3, 2)
e

tensor([[[ 0.8146, -0.4564],
         [-0.1064,  0.2428],
         [-0.9425, -0.5104]],

        [[-0.1018, -0.7361],
         [-0.4267, -2.2279],
         [-0.7795,  2.7883]]])

In [55]:
e.permute(2, 1, 0)

tensor([[[ 0.8146, -0.1018],
         [-0.1064, -0.4267],
         [-0.9425, -0.7795]],

        [[-0.4564, -0.7361],
         [ 0.2428, -2.2279],
         [-0.5104,  2.7883]]])

In [2]:
import torch

a = torch.zeros(10, 2, 3)
a.shape

torch.Size([10, 2, 3])

In [5]:
a = a.unsqueeze(1)
a.shape

torch.Size([10, 1, 2, 3])

In [2]:
import os, shutil
for d in os.listdir('E:\\PFE_DATA\\prepared_data\\test'):
    l = len(os.listdir('E:\\PFE_DATA\\prepared_data\\test\\' + d))
    if l != 10:
        print(d, l)
        shutil.rmtree('E:\\PFE_DATA\\prepared_data\\test\\' + d)

original_sequences_youtube_c23_videos_082_face1_10_0 1
